# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../WeatherPy/city_data.csv")
vacation_df.head()

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Cape Town,ZA,0,1619798116,52,-33.9258,18.4232,64.40,19.57
1,Hithadhoo,MV,37,1619798682,73,-0.6000,73.0833,84.09,7.65
2,Freeport,US,20,1619798408,42,40.6576,-73.5832,66.99,23.02
3,Qaanaaq,GL,53,1619798104,64,77.4840,-69.3632,32.34,7.07
4,Talnakh,RU,25,1619798224,84,69.4865,88.3972,19.40,3.38


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = vacation_df[["Latitude", "Longitude"]].astype(float)
humidity = vacation_df["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(height='420px'))

In [4]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
ideal_df = pd.DataFrame(vacation_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])
temp = (ideal_df["Max Temp"]<=80) & (ideal_df["Max Temp"]>=70)
wind = (ideal_df["Wind Speed"]<=10)
clouds = (ideal_df["Cloudiness"]==0)
ideal_df[temp & wind & clouds]



In [60]:
ideal_df = vacation_df.loc[(vacation_df["Max Temp"]>=70) & (vacation_df["Max Temp"]<=80) & (vacation_df["Wind Speed"]<=10) & (vacation_df["Cloudiness"]==0), :]
ideal_df = ideal_df.dropna(how='any')
ideal_df.reset_index(inplace=True)
del ideal_df['index']
ideal_df
#no null values

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Cacequi,BR,0,1619798709,37,-29.8836,-54.8250,77.11,8.25
1,Saint-Philippe,RE,0,1619798719,83,-21.3585,55.7679,77.00,4.61
2,Kumul,CN,0,1619798866,12,42.8000,93.4500,76.57,3.31
3,Tacuarembó,UY,0,1619798891,46,-31.7333,-55.9833,72.90,8.28
4,Korla,CN,0,1619798913,15,41.7597,86.1469,76.51,4.03
5,Saint-Joseph,RE,0,1619798934,83,-21.3667,55.6167,77.00,4.61
6,Guerrero Negro,MX,0,1619798953,44,27.9769,-114.0611,73.87,8.32
7,Saint-Pierre,RE,0,1619799029,83,-21.3393,55.4781,77.00,4.61
8,Chui,UY,0,1619799071,47,-33.6971,-53.4616,70.34,7.87
9,San Quintín,MX,0,1619799092,38,30.4833,-115.9500,75.61,5.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [72]:
hotel_df = []

for x in range(len(ideal_df)):
    lat = ideal_df.loc[x]['Latitude']
    lng = ideal_df.loc[x]['Longitude']
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    
    try:
        hotel_df.append(response['results'][0]['name'])
    except:
        hotel_df.append("")

In [73]:
ideal_df["Hotel Name"] = hotel_df
ideal_df

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Cacequi,BR,0,1619798709,37,-29.8836,-54.8250,77.11,8.25,Cacequi
1,Saint-Philippe,RE,0,1619798719,83,-21.3585,55.7679,77.00,4.61,Saint-Philippe
2,Kumul,CN,0,1619798866,12,42.8000,93.4500,76.57,3.31,Hami
3,Tacuarembó,UY,0,1619798891,46,-31.7333,-55.9833,72.90,8.28,Tacuarembo
4,Korla,CN,0,1619798913,15,41.7597,86.1469,76.51,4.03,Bayingolin Mongol Autonomous Prefecture
5,Saint-Joseph,RE,0,1619798934,83,-21.3667,55.6167,77.00,4.61,Saint-Joseph
6,Guerrero Negro,MX,0,1619798953,44,27.9769,-114.0611,73.87,8.32,Guerrero Negro
7,Saint-Pierre,RE,0,1619799029,83,-21.3393,55.4781,77.00,4.61,Saint Pierre
8,Chui,UY,0,1619799071,47,-33.6971,-53.4616,70.34,7.87,Chuy
9,San Quintín,MX,0,1619799092,38,30.4833,-115.9500,75.61,5.91,Lázaro Cárdenas


In [74]:
ideal_df = ideal_df.dropna(how='any')
ideal_df

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Cacequi,BR,0,1619798709,37,-29.8836,-54.8250,77.11,8.25,Cacequi
1,Saint-Philippe,RE,0,1619798719,83,-21.3585,55.7679,77.00,4.61,Saint-Philippe
2,Kumul,CN,0,1619798866,12,42.8000,93.4500,76.57,3.31,Hami
3,Tacuarembó,UY,0,1619798891,46,-31.7333,-55.9833,72.90,8.28,Tacuarembo
4,Korla,CN,0,1619798913,15,41.7597,86.1469,76.51,4.03,Bayingolin Mongol Autonomous Prefecture
5,Saint-Joseph,RE,0,1619798934,83,-21.3667,55.6167,77.00,4.61,Saint-Joseph
6,Guerrero Negro,MX,0,1619798953,44,27.9769,-114.0611,73.87,8.32,Guerrero Negro
7,Saint-Pierre,RE,0,1619799029,83,-21.3393,55.4781,77.00,4.61,Saint Pierre
8,Chui,UY,0,1619799071,47,-33.6971,-53.4616,70.34,7.87,Chuy
9,San Quintín,MX,0,1619799092,38,30.4833,-115.9500,75.61,5.91,Lázaro Cárdenas


In [79]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Latitude", "Longitude"]]
locations

,Latitude,Longitude
0,-29.8836,-54.8250
1,-21.3585,55.7679
2,42.8000,93.4500
3,-31.7333,-55.9833
4,41.7597,86.1469
5,-21.3667,55.6167
6,27.9769,-114.0611
7,-21.3393,55.4781
8,-33.6971,-53.4616
9,30.4833,-115.9500


In [83]:
# Add marker layer ontop of heat map
markers= gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))